# VQE: Computational cost v.s. performance

Today, the main challenge in using VQE for chemistry is finding a balance between the required computational costs and the accuracy of the obtained results.
In other words, a circuit ansatz generally becomes better at modelling the electronic wave function as more gates are included in the ansatz.
However, the circuit depth that can be implemented on an actual physical quantum computer is still limited by the level of quantum hardware today.

**(ZC note: Not a nice sentence...)**
Hence, the question of how to reduce the circuit depth without losing too much accuracy is an active area of research today.
We will explore the issue of circuit complexity v.s. accuracy using a slightly larger molecule, LiH, in this notebook.


#### Sidenote: Chemical accuracy

- Energy available at room temperature:
    - RT = 8.31 J mol$^{-1}$ K$^{-1}$ * 300 K = 2.5 kJ/mol

In [ ]:
import numpy as np

import openfermion

# Molecule class to store the molecular integrals and other relevant quantities
from qibochem.driver import Molecule

In [ ]:
lih = Molecule(xyz_file='lih.xyz')
lih.run_pyscf()

### HF Embedding


It is known that the core-shell electrons in a molecule (i.e. the electrons occupying the 1s orbitals on atoms larger than H or He) only affects a molecule's physical and chemical properties weakly.
Hence, one simplifiying approximation is to 'freeze' these core electrons, leaving them out of the quantum simulation, while still partially including their effect on the other electrons.

More specifically, the electrons are embedded in "a classically computed environment obtained at the Hartree-Fock (HF) ...  level of theory", and is known as [Hartree-Fock embedding](https://dx.doi.org/10.1063/5.0029536).
Similarly, the virtual (unoccupied) orbitals that are not expected to play a significant part in chemical bonding can also be removed from the quantum simulation.

***

The `hf_embedding` class function fills in the class attributes related to HF embedding.
We demonstrate how to use it in Qibochem in the cells below: 

In [ ]:
# First, a rough guide as to how to choose what orbitals to keep/remove
print(f"HF orbital energies: {lih.eps}")

The energy of the first molecular orbital is significantly lower than the others; this is the 1s orbital on Li, and can be removed.
We can also remove molecular orbitals 4 and 5 by symmetry considerations.
Aligning the LiH molecule along the z-axis, these two orbitals are the $2p_x$ and $2p_y$ orbitals, which are orthogonal to the Li-H bond.
Hence, they should not have much of an effect on the bonding in LiH, and can be removed.

In [ ]:
# These are the class attributes that will be filled after running hf_embedding:

# Before turning on embedding:
# print(lih.embed_oei)
# print(lih.embed_tei)
print("Inactive Fock energy:", lih.inactive_energy)
print("Number of active molecular orbitals:", lih.n_active_orbs)
print("Number of active electrons:", lih.n_active_e)

In [ ]:
# After running hf_embedding:
active = [1, 2, 5] # Python indexes from 0
lih.hf_embedding(active=active)

# print(lih.embed_oei)
# print(lih.embed_tei)
print("Inactive Fock energy:", lih.inactive_energy)
print("Number of active molecular orbitals:", lih.n_active_orbs)
print("Number of active electrons:", lih.n_active_e)

Compare the difference in the length of the molecular fermionic Hamiltonian with and without applying embedding:

In [ ]:
# Without embedding:
fermionic_hamiltonian = lih.hamiltonian("f", oei=lih.oei, tei=lih.tei, constant=0.0)
# We needed to include the oei/tei/constant arguments because the .hamiltonian() function will check if embedding has been carried out and defines the 
# molecular Hamiltonian accordingly.

embedded_fermionic_hamiltonian = lih.hamiltonian("f")

print(f"Number of terms without HF embedding: {len(fermionic_hamiltonian.terms)}")
print(f"Number of terms with HF embedding: {len(embedded_fermionic_hamiltonian.terms)}")

The number of terms in the fermionic Hamiltonian decreased by more than 5 times!

***

Next, we check that the electronic energy should be largely unchanged after applying HF embedding by diagonalizing both Hamiltonians exactly:

Warning: Diagonalizing the Hamiltonian exactly is only possible for small systems!

In [ ]:
from scipy.sparse import linalg

hamiltonian_matrix = openfermion.get_sparse_operator(fermionic_hamiltonian)
eigenvalues, _ = linalg.eigsh(hamiltonian_matrix, k=6, which="SA")

exact_result = eigenvalues[0]


In [ ]:
embedded_hamiltonian_matrix = openfermion.get_sparse_operator(embedded_fermionic_hamiltonian)
eigenvalues, _ = linalg.eigsh(embedded_hamiltonian_matrix, k=6, which="SA")

embedded_exact_result = eigenvalues[0]


In [ ]:
print(f"Exact result without embedding: {exact_result}")
print(f"   Exact result with embedding: {embedded_exact_result}")

So, we see that reducing the number of terms in the molecular Hamiltonian by about four-fifth's only leads to a difference between the exact results for both Hamiltonians of about only 1e-4!

---

### Encoding the molecular fermion Hamiltonian

In the previous notebook, we used the Jordan-Wigner transformation to map the occupancy of a spin-orbital to the Pauli Z state of a qubit directly.
We will use the Brayvi-Kitaev map in this notebook, which produces less terms in the qubit Hamiltonian for larger systems.

In [ ]:
jw_mol_hamiltonian = openfermion.jordan_wigner(embedded_fermionic_hamiltonian)
bk_mol_hamiltonian = openfermion.bravyi_kitaev(embedded_fermionic_hamiltonian)

print(f"Number of terms (JW): {len(jw_mol_hamiltonian.terms)}")
print(f"Number of terms (BK): {len(bk_mol_hamiltonian.terms)}")

Note: The threshold for the BK mapping outperforming JW is about 16 qubits (IIRC), so we don't see a difference for the small LiH system

## Ansatz: Unitary coupled-cluster

### Quantum Chemistry: Post-Hartree-Fock methods

- Limitations of Hartree-Fock:
    - No electron correlation, i.e. electrons don't 'see' the other individual electrons, and only 'see' the overall 'electron cloud'
    - Cannot fully describe a chemical reaction
        - E.g. $N_2$ as the N atoms are separated
            - Hartree-Fock cannot completely describe which electrons 'go' to which of the two N atoms

- Post-Hartree-Fock methods:
    - Hartree-Fock wave function used as a starting point, then aim to improve the treatment of electron correlation
    - E.g. allow the electrons to be placed into higher energy molecular orbitals, and include these cases in the treatment of the ground state wave function

### Unitary Coupled-Cluster (UCC):

$$
\exp(\hat{T} - \hat{T}^\dagger) \lvert \Psi_{HF} \rangle
$$

- Excitation operator $\hat{T}$ acts on the Hartree-Fock wave function 
    1. Annihilates one or more electrons from the occupied orbitals
    2. Creates the same number of annihilated electrons in the unoccupied orbitals
-  Typically only consider up to 2 electrons being excited
    - **S**ingles and **D**oubles $\implies$ UCCSD

$$
\hat{T} = \hat{T}_1 + \hat{T}_2
$$

$$
\text{with } \hat{T}_1 = \sum_{i \in \text{occ.} \\ a \in \text{unocc.}} \hat{a}_a^\dagger \hat{a}_i
$$

$$
\hat{T}_2 = \sum_{i,j \in \text{occ.} \\ a,b \in \text{unocc.}} \hat{a}_b^\dagger \hat{a}_a^\dagger \hat{a}_j \hat{a}_i
$$


#### UCCSD on a quantum computer

1. As was done for the molecular Hamiltonian, apply a fermion-to-qubit mapping on the fermionic creation/annihilation operators in the UCC operator

$$
\exp \left( \sum_{i} ( \hat{T}_i - \hat{T}_i^\dagger ) \right)
= \exp ( \sum_{i}  \hat{P}_i  ) 
\text{, where $\hat{P}_i$ is a Pauli string}
$$

2. Result: Exponential of a sum of Pauli strings 
    - Apply Trotterization approximation, i.e. exponential of sum $\approx$ product of exponentials

$$
\exp ( \sum_{i}  \hat{P}_i  ) \approx \prod_i (\exp \hat{P}_i )
$$

3. After Trotterization, apply the exp$(\hat{P}_i)$ term corresponding to a singles/doubles excitation to the quantum circuit
    - Note: Exact order of excitations being applied has been found to affect final optimized energy

In [ ]:
# UCCSD ansatz with Qibochem
from qibochem.ansatz import hf_circuit, ucc_circuit

In [ ]:
# Example: UCC with LiH

# Define a double excitation from the first two spin-orbitals (after embedding!) to the next two
excitation = [0, 1, 2, 3]

# Build the UCC circuit with the JW map
# UCC is a post-HF method, hence the need to start with a HF ansatz
circuit = hf_circuit(lih.n_active_orbs, lih.n_active_e)
circuit += ucc_circuit(n_qubits=lih.n_active_orbs, excitation=excitation)
print(circuit.draw())
print()
print("Circuit summary:")
print(circuit.summary())

In [ ]:
# Build the UCC circuit with the BK map
# UCC is a post-HF method, hence the need to start with a HF ansatz
circuit = hf_circuit(lih.n_active_orbs, lih.n_active_e, ferm_qubit_map='bk')
circuit += ucc_circuit(n_qubits=lih.n_active_orbs, excitation=excitation, ferm_qubit_map='bk')

print(circuit.draw())
print()
print("Circuit summary:")
print(circuit.summary())

- Notes:
    - Both circuits can be compiled further, e.g. consecutive --H--H-- gates on the same qubit can be cancelled out
    - JW mapping requires more gates (greater circuit depth), but only needs nearest-neighbour circuit connectivity
    - BK mapping has a lower circuit depth, but may require additional SWAP gates depending on the quantum hardware
    
Next, we confirm that both circuits are equivalent:

In [ ]:
# Convert the JW/BK qubit Hamiltonians to SymbolicHamiltonians
from qibochem.driver.hamiltonian import qubit_to_symbolic_hamiltonian

jw_sym_hamiltonian = qubit_to_symbolic_hamiltonian(jw_mol_hamiltonian)
bk_sym_hamiltonian = qubit_to_symbolic_hamiltonian(bk_mol_hamiltonian)

In [ ]:
from qibochem.measurement import expectation

In [ ]:
# Run test 3 times
for _ in range(3):
    # Random initialization
    theta = np.random.rand(1)
    print(f"Random initial theta: {theta[0]}") # Theta should be an array of dim=8

    # JW circuit
    jw_circuit = hf_circuit(lih.n_active_orbs, lih.n_active_e)
    jw_circuit += ucc_circuit(n_qubits=lih.n_active_orbs, excitation=excitation, theta=theta)

    # BK circuit
    bk_circuit = hf_circuit(lih.n_active_orbs, lih.n_active_e, ferm_qubit_map='bk')
    bk_circuit += ucc_circuit(n_qubits=lih.n_active_orbs, excitation=excitation, ferm_qubit_map='bk', theta=theta)

    # Energy expectation value for both circuits
    print(f"JW-UCC energy: {expectation(jw_circuit, jw_sym_hamiltonian)}")
    print(f"BK-UCC energy: {expectation(bk_circuit, bk_sym_hamiltonian)}")
    print()

For the same $\theta$, both circuits yield the same energy expectation value.
We are more confident that the circuits (when used with their respective Hamiltonians) are equivalent.

The next step is then to run a VQE to minimize the electronic energy of our LiH system.


In [ ]:
from qibo import models
from qibo.optimizers import optimize

In [ ]:
%%time

# Example: Using the VQE class from Qibo:
bk_circuit = hf_circuit(lih.n_active_orbs, lih.n_active_e, ferm_qubit_map='bk')
bk_circuit += ucc_circuit(n_qubits=lih.n_active_orbs, excitation=excitation, ferm_qubit_map='bk')

# Get the number of paramerized gates
n_param_gates = len(bk_circuit.get_parameters())

# Create the VQE model
vqe = models.VQE(bk_circuit, bk_sym_hamiltonian)

# Optimize starting from a random guess for the variational parameters
initial_parameters = np.random.uniform(0, 2*np.pi,
                                        n_param_gates)

best, params, extra = vqe.minimize(initial_parameters, method='BFGS')# , compile=False) 

In [ ]:
print("Results:")
print(f"{' ':>4}VQE energy: {best}\n")

print(f"{' ':>5}HF energy: {lih.e_hf}")
# Exact groundstate energy of H2 as a reference
print(f"Exact solution: {embedded_exact_result}")

The VQE energy only differs from the initial HF energy because the current circuit ansatz only includes a single excitation.
But before we extend the circuit ansatz by including more excitations, there is an additional step that we can take to speed up the optimization.

---

Ths problem with using the `VQE` class directly from Qibo is that it ignores the fact that the absolute value of all 8 parameters should be equal.

In [ ]:
print(f"Are all the parameters equal? {len(np.unique(np.abs(params), return_counts=True)[0])==1}")
print(f"\nOptimized parameters: {np.abs(params)}")

Hence, it's probably more correct (and potentially faster) to define a [custom variational ciruit](https://qibo.readthedocs.io/en/stable/code-examples/advancedexamples.html#how-to-write-a-custom-variational-circuit-optimization) instead.

In [ ]:
%%time

bk_circuit = hf_circuit(lih.n_active_orbs, lih.n_active_e, ferm_qubit_map='bk')
bk_circuit += ucc_circuit(n_qubits=lih.n_active_orbs,
                          excitation=excitation,
                          ferm_qubit_map='bk',
                         )
# The sign of the individual circuit parameters depends on the fermion-to-qubit map used, and is given here for convenience
coeff_dict = {
    "bk": np.array([-0.25, -0.25, 0.25, 0.25, 0.25, 0.25, -0.25, -0.25]),
    "jw": np.array([-0.25, 0.25, 0.25, 0.25, -0.25, -0.25, -0.25, 0.25])
}

# Custom loss function; i.e. electronic energy of LiH
def electronic_energy(parameter):
    coeffs = coeff_dict["bk"]
    # Convert a single value to a array with dimension=n_param_gates
    parameters = np.repeat(parameter, len(coeffs))
    # Multiply by the correct coeffs
    parameters *= coeffs

    bk_circuit.set_parameters(parameters)

    return expectation(bk_circuit, bk_sym_hamiltonian)


# Optimize starting from a random guess for the variational parameters
theta = np.random.rand(1)

# Perform optimization
best_v2, params_v2, extra_v2 = optimize(electronic_energy, theta)# , method='BFGS')

In [ ]:
print("Results:")
print(f"{' ':>4}VQE energy: {best_v2} (Current run)")
print(f"{' ':>4}VQE energy: {best_v2} (Earlier run)\n")

print(f"{' ':>5}HF energy: {lih.e_hf}")
# Exact groundstate energy of H2 as a reference
print(f"Exact solution: {embedded_exact_result}")

In [ ]:
print(f"Are all the parameters equal? {len(np.unique(params_v2, return_counts=True)[0])==1}")
print(f"\nOptimized parameters: {params_v2}")

Although both results are the same, the second approach, of using a custom variational circuit should be a lot faster!

***

#### Full VQE-UCCSD with LiH

The above example was for a single double excitation.
Next, we will use the full UCCSD ansatz to find the electronic energy of LiH, and assess its performance.

In [ ]:
# Generate all single and double excitations first
s_excitations = [[_e, _orb] for _e in range(lih.n_active_e)
                 for _orb in range(lih.n_active_e, lih.n_active_orbs)
                 if (_e + _orb) % 2 == 0 # Spin-conservation
                ]
d_excitations = [[_e1, _e2, _orb1, _orb2]
                 for _e1 in range(lih.n_active_e)
                 for _e2 in range(_e1+1, lih.n_active_e)
                 for _orb1 in range(lih.n_active_e, lih.n_active_orbs)
                 for _orb2 in range(_orb1+1, lih.n_active_orbs)
                 # Spin-conservation
                 if (_e1 + _e2 + _orb1 + _orb2) % 2 == 0
                ]


print("Single excitations:", s_excitations)
print("Double excitations:", d_excitations)

In [ ]:
# Empirically, the pair excitations should go first, so we re-order d_excitation slightly
d_excitations = sorted(d_excitations,
                      key=lambda x: 10*(x[-2] % 2) + x[-1] - x[-2] # Very contrived lambda for sorting
                     )
print("Double excitations (after sorting):", d_excitations)

# Then we pair the single excitations involving the same set of molecular orbitals together (manually)
s_excitations = [s_excitations[0], s_excitations[2], s_excitations[1], s_excitations[3]]

# Combine the single and double excitations into a single list
# Again, empirically, putting the double excitations first is seems to be stable
all_excitations = d_excitations + s_excitations
print("\nAll excitations:", all_excitations)
n_excitations = len(all_excitations)
print("\nNumber of excitations:", n_excitations)

In [ ]:
# Now build the full UCCSD circuit
bk_uccsd_circuit = hf_circuit(lih.n_active_orbs, lih.n_active_e, ferm_qubit_map='bk')
# Add gates corresponding to each excitation
for _ex in all_excitations:
    bk_uccsd_circuit += ucc_circuit(n_qubits=lih.n_active_orbs,
                              excitation=_ex,
                              ferm_qubit_map='bk',
                             )
    
# Visualise the resultant circuit
print(bk_uccsd_circuit.draw())

In [ ]:
%%time

# Lastly, the actual optimization

# Redefine the custom loss function for the full UCCSD
def uccsd_electronic_energy(parameters):
    # Coefficients for the circuit parameters for single/double excitations with the BK fermion to qubit map
    bk_coeffs = {1: (-1.0, -1.0), 2: (-0.25, -0.25, 0.25, 0.25, 0.25, 0.25, -0.25, -0.25)}

    all_parameters = []
    # Need to iterate through each excitation this time
    for _ex, parameter in zip(all_excitations, parameters):
        coeffs = bk_coeffs[len(_ex) // 2]
        # Convert a single value to a array with dimension=n_param_gates
        ucc_parameter = np.repeat(parameter, len(coeffs))
        # Multiply by coeffs
        ucc_parameter *= coeffs
        all_parameters.append(ucc_parameter)

    # Flatten all_parameters into a single list
    all_parameters = [_x for _param in all_parameters for _x in _param]
    bk_uccsd_circuit.set_parameters(all_parameters)

    return expectation(bk_uccsd_circuit, bk_sym_hamiltonian)


# Optimize starting from a random guess for the variational parameters
# thetas = np.random.rand(n_excitations)
# Starting from zeros is possible too!
thetas = np.zeros(n_excitations)
# # Perform optimization
uccsd_best, uccsd_params, uccsd_extra = optimize(uccsd_electronic_energy, thetas, method='BFGS')

In [ ]:
print("Results:")
print(f"{' ':>5}HF energy: {lih.e_hf}")
print(f"{' ':>4}VQE energy: {best_v2} (Only one double excitation)\n")

print(f"{' ':>4}VQE energy: {uccsd_best} (Current run)")
# Exact groundstate energy of H2 as a reference
print(f"Exact solution: {embedded_exact_result}")

The result should be better than the run with only one double excitation, and should also come quite close to the exact solution.

Lastly, take note of the circuit statistics (printed below).
Based on the length of the circuit, the probability of it running completely should be unfortunately quite low...


In [ ]:
print(bk_uccsd_circuit.summary())

## Ansatz: Hardware-efficient

As per its name, the [hardware-efficient ansatz](https://www.nature.com/articles/nature23879) was designed such that it could be used on current quantum hardware.
Essentially, it is a general ansatz that can be used in a variety of applications.
We have already used it in the first notebook for $H_2$, and will compare its performance for $LiH$ against the UCCSD ansatz here.

In [ ]:
from qibochem.ansatz import he_circuit

In [ ]:
# Let's start with a single layer first
h_circuit = he_circuit(n_qubits=lih.n_active_orbs, n_layers=1, coupling_gates="CNOT")
print(circuit.draw())

In [ ]:
# Get the number of paramerized gates
n_param_gates = len(h_circuit.get_parameters())

# No restrictions for the parameters; can use the VQE class directly
he_vqe = models.VQE(h_circuit, bk_sym_hamiltonian)

In [ ]:
%%time

# Optimize starting from a random guess for the variational parameters
initial_parameters = np.random.uniform(0, 2*np.pi,
                                        n_param_gates)
he_best, he_params, he_extra = he_vqe.minimize(initial_parameters)# method='BFGS', compile=False) 

In [ ]:
print("Results:")
print(f"{' ':>5}HF energy: {lih.e_hf}")
# print(f"{' ':>4}VQE energy: {uccsd_best} (UCCSD)\n")

print(f"{' ':>4}VQE energy: {he_best} (Hardware-efficient)")
# Exact groundstate energy of H2 as a reference
print(f"Exact solution: {embedded_exact_result}")

### Barren plateau problem:

For a single layer, the quality of results is largely arbitrary.
Depending on the initial set of random parameters, we might reach some local minimum (e.g. HF energy), and might never obtain the true global minimum, or even a solution close to it.
(The time taken for the optimization can also vary.)

***

To improve the performance of the hardware-efficient ansatz, we need to add more layers of entangling gates, to improve the expressibility of the ansatz.

In [ ]:
# Try using 3 layers this time...
h3_circuit = he_circuit(n_qubits=lih.n_active_orbs, n_layers=3, coupling_gates="CNOT")

print(h3_circuit.draw())

In [ ]:
# Get the number of paramerized gates
n_param_gates = len(h3_circuit.get_parameters())

# No restrictions for the parameters; can use the VQE class directly
h3_vqe = models.VQE(h3_circuit, bk_sym_hamiltonian)

In [ ]:
%%time

# Optimize starting from a random guess for the variational parameters
initial_parameters = np.random.uniform(0, 2*np.pi,
                                        n_param_gates)
he3_best, he3_params, he3_extra = h3_vqe.minimize(initial_parameters)# method='BFGS', compile=False)

In [ ]:
print("Results:")
print(f"{' ':>5}HF energy: {lih.e_hf}")
# print(f"{' ':>4}VQE energy: {uccsd_best} (UCCSD)\n")

print(f"{' ':>4}VQE energy: {he3_best} (Hardware-efficient)")
# Exact groundstate energy of H2 as a reference
print(f"Exact solution: {embedded_exact_result}")

Again, the obtained VQE energy might not be anywhere near the correct result despite the increase in the number of layers.
A further increase in the number of layers improves the expressibility of the ansatz, but this might worsen the barren plateau problem.

Either way, the reliability of the hardware-efficient ansatz can be said to be poorer than a chemistry-based ansatz such as UCCSD.
However, because it has a significantly lower gate depth and complexity (see below cell), it can actually be implemented on the current quantum computers.


In [ ]:
print(h3_circuit.summary())

An active area of research today is thus finding a balance between the (quantum) computational cost and the resultant accuracy:

- Improving the ansatz:
    - Hardware-efficient:
        - How to improve the reliability, without increasing the circuit depth too much?
    - Chemistry-based:
        - How to reduce the circuit depth of these methods efficiently?